# Wavelength calibration

In [1]:
import numpy as np
import glob
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'
import sys
codefolder = '../../../../crispy'
if codefolder not in sys.path: sys.path.append(codefolder)
from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.params import Params
codefolder = '../../../crispy'
par = Params(codefolder)

### Loading crispy's dependencies

### Generate monochromatic wavelengths

In [2]:
# Suppose we create a folder in crispy/ReferenceFiles/ called wavecalR50_770_test
par.wavecalDir = par.prefix+'/wavecalR50_660/'
# construct monochromatic wavelengths, doesn't really matter which wavelengths
lamlist = np.arange(590,735,10)


In [3]:
# construct images for each wavelength
par.savePoly = True
from crispy.IFS import createWavecalFiles
createWavecalFiles(par,lamlist)

crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths
crispy - INFO - Writing data to ../../../crispy/SimResults/detectorFramepoly.fits
crispy - INFO - Done.
crispy - INFO - Performance: 4 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_660/det_590.fits
crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths
crispy - INFO - Writing data to ../../../crispy/SimResults/detectorFramepoly.fits
crispy - INFO - Done.
crispy - INFO - Performance: 4 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_660/det_600.fits
crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming

### Loading wavelengths/filelists

In [10]:
#par.lamlist = np.loadtxt(par.wavecalDir + "lamsol.dat")[:, 0]
par.lamlist=lamlist
par.filelist = np.sort(glob.glob(par.wavecalDir + "det*.fits"))

### Run calibration. Choose parameters wisely.

In [11]:
from crispy.tools.wavecal import buildcalibrations
buildcalibrations(par,
                  order=3, # Order of polynomial map of the PSFLets (between 1 and 5)
                  upsample=10, # Upsampling factor of the PSFLets, needs to be at least 5
                  nsubarr=1, # How many zones on the detector have their own PSFLets? 
                  genwavelengthsol=True, # Compute wavelength at the center of all pixels (need to do this at least once)
                  makehiresPSFlets=True, # Compute hires PSFLets (usually False)
                  makePolychrome=True,
                  apodize=False,
                  parallel=True) # Use all computer cores

crispy - INFO - Building calibration files, placing results in ../../../crispy/ReferenceFiles/wavecalR50_660/
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_660/det_590.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_660/det_590.fits
crispy - INFO - Initializing PSFlet location transformation coefficients
crispy - INFO - Performing initial optimization of PSFlet location transformation coefficients for frame ../../../crispy/ReferenceFiles/wavecalR50_660/det_590.fits
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ../../../crispy/ReferenceFiles/wavecalR50_660/det_590.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_660/det_600.fits
crispy - INFO - Initializing transformation coefficients with previous values
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ../../../cris

Traceback (most recent call last):
  File "/Users/mrizzo/anaconda/lib/python2.7/multiprocessing/queues.py", line 268, in _feed
    send(obj)
IOError: [Errno 32] Broken pipe
IOError: [Errno 32] Broken pipe


crispy - INFO - Reduced cube will have 11 wavelength bins
crispy - INFO - Making polychrome cube
crispy - INFO - Saving polychrome cube
crispy - INFO - Saving wavelength calibration cube
crispy - INFO - Total time elapsed: 286 s


In [12]:
from crispy.tools.reduction import calculateWaveList
lam_midpts,lam_endpts = calculateWaveList(par,method='lstsq')
print('Wavelengths at midpoints for lstsq (nm): ',lam_midpts)
print('Wavelengths at endpoints for lstsq (nm): ',lam_endpts)
lam_midpts,lam_endpts = calculateWaveList(par,method='optext')
print('Wavelengths at midpoints for optext (nm): ',lam_midpts)
print('Wavelengths at endpoints for optext (nm): ',lam_endpts)

crispy - INFO - Reduced cube will have 11 wavelength bins
('Wavelengths at midpoints for lstsq (nm): ', array([ 596.36363636,  609.09090909,  621.81818182,  634.54545455,
        647.27272727,  660.        ,  672.72727273,  685.45454545,
        698.18181818,  710.90909091,  723.63636364]))
('Wavelengths at endpoints for lstsq (nm): ', array([ 590.        ,  602.72727273,  615.45454545,  628.18181818,
        640.90909091,  653.63636364,  666.36363636,  679.09090909,
        691.81818182,  704.54545455,  717.27272727,  730.        ]))
crispy - INFO - Reduced cube will have 11 wavelength bins
('Wavelengths at midpoints for optext (nm): ', array([ 596.36363636,  609.09090909,  621.81818182,  634.54545455,
        647.27272727,  660.        ,  672.72727273,  685.45454545,
        698.18181818,  710.90909091,  723.63636364]))
('Wavelengths at endpoints for optext (nm): ', array([ 590.        ,  602.72727273,  615.45454545,  628.18181818,
        640.90909091,  653.63636364,  666.36363636, 

In [8]:
print len(par.lamlist)

15
